In [ ]:
// Mes imports de librairies externes

USE {
    repositories {
        //google()
        // à exécuter en dehors du proxy sinon KO
        mavenCentral()
        //maven("https://artifactory.mycloud.intrabpce.fr/artifactory/g-android-maven-proxy/")
        //maven("https://artifactory.mycloud.intrabpce.fr/artifactory/maven-third-party-android-libs/")
    }
    dependencies {
        val ktor_version = "2.2.4"
        implementation("io.ktor:ktor-client-apache-jvm:$ktor_version")
        implementation("io.ktor:ktor-serialization-kotlinx-json:$ktor_version")
    }
}

In [ ]:

import io.ktor.client.*
import io.ktor.client.request.*
import io.ktor.client.statement.*
import kotlinx.coroutines.runBlocking

val client = HttpClient()
runBlocking {
    client.get("https://google.com").bodyAsText()
 }

In [33]:
import io.ktor.client.engine.apache.*
import io.ktor.client.*
import io.ktor.client.plugins.*
import io.ktor.client.request.*
import io.ktor.client.statement.*
import io.ktor.http.*
import kotlinx.serialization.SerialName
import kotlinx.serialization.Serializable
import kotlinx.serialization.decodeFromString
import kotlinx.serialization.json.Json

@Serializable
data class SearchProductList(
    @SerialName("products") val products: List<ProductDetail>? = null
)

@Serializable
data class Product(
    @SerialName("code") val productId: String? = null,
    @SerialName("product") val productDetail: ProductDetail? = null,
)

@Serializable
data class ProductDetail(
    @SerialName("code") val code: String = "",
    @SerialName("product_name_fr") val label: String = "",
    @SerialName("ingredients_text") val ingredientsText: String = "",
    @SerialName("ingredients_text_fr") val ingredientsTextFr: String = "",
    @SerialName("generic_name_fr") val genericName: String = "",
    @SerialName("image_url") val imageUrl: String = "",
    @SerialName("nutrition_grade_fr") val nutriScore: String = "",
    @SerialName("ecoscore_grade") val ecoScore: String = "",
    @SerialName("ecoscore_score") val ecoScoreValue: String = ""
)

class NutriScoreApp() {
    val jsonFormatter = Json { ignoreUnknownKeys = true }

    suspend fun getProduct(productId: String): Product {
        val client = getHttpClient()
        val getProductUrl = "https://fr.openfoodfacts.org/api/v0/produit/$productId.json"
        val productText = client.get(getProductUrl + FIELDS).bodyAsText()
        val product: Product = jsonFormatter.decodeFromString(productText)
        client.close()
        return product
    }

    suspend fun searchProductsJson(search: String): String {
        val client = getHttpClient()
        val searchProductUrl = "https://ssl-api.openfoodfacts.org/cgi/search.pl?search_simple=1&amp;json=1&amp;action=process&amp;sort_by=unique_scans_n"
        val extraParams = "&amp;search_terms=${search.encodeURLPath()}&amp;page=1"
        val searchText = client.get(searchProductUrl + FIELDS + extraParams).bodyAsText()
        return searchText
    }

    suspend fun searchProducts(search: String): List<ProductDetail> {
        val searchResult: SearchProductList = jsonFormatter.decodeFromString(searchProductsJson(search))
        client.close()
        return searchResult.products.orEmpty()
    }

    private fun getHttpClient() = HttpClient {
            install(HttpTimeout) {
                requestTimeoutMillis = 30000
            }
        }

    companion object {
        const val FIELDS = "&amp;fields=selected_images,image_url,product_name,brands,quantity,code,nutrition_grade_fr,ecoscore_grade,product_name_fr,nova_groups,allergens_tags,additives_tags,ingredients_from_palm_oil_n,carbon-footprint-from-known-ingredients_product,carbon-footprint-from-meat-or-fish_product,nutriments"
    }
}

In [ ]:
import kotlinx.coroutines.runBlocking

// Exemple de produit :
// Gâteaux au chocolat : 3175681134935
runBlocking {
    val nutriScoreApp = NutriScoreApp()
    val product = nutriScoreApp.getProduct("3175681134935")
    HTML(
        "<h2>${product.productDetail?.label}</h2><br>" +
                "<img src='${product.productDetail?.imageUrl}'/>"
    )
}


In [34]:
import kotlinx.coroutines.runBlocking

// Exemple de recherche :
runBlocking {
    val search = "pate à tartiner noisette"
    val nutriScoreApp = NutriScoreApp()
    val searchResult = nutriScoreApp.searchProducts(search)
    println(searchResult)

    var searchResultToDisplay: String = ""
    searchResult.forEach {
        searchResultToDisplay += "<h2>${it.label}</h2><br>" +
        if (it.imageUrl.isNotEmpty()) "<img src='${it.imageUrl}'/>" else ""
    }
    HTML(searchResultToDisplay)
}

[ProductDetail(code=3017620422003, label=Pâte à tartiner Nutella noisettes et cacao, ingredientsText=, ingredientsTextFr=, genericName=, imageUrl=https://images.openfoodfacts.org/images/products/301/762/042/2003/front_en.502.400.jpg, nutriScore=e, ecoScore=d, ecoScoreValue=), ProductDetail(code=3017620425035, label=Nutella, ingredientsText=, ingredientsTextFr=, genericName=, imageUrl=https://images.openfoodfacts.org/images/products/301/762/042/5035/front_en.397.400.jpg, nutriScore=e, ecoScore=d, ecoScoreValue=), ProductDetail(code=8001505005592, label=Nocciolata Pâte à tartiner au cacao et noisettes, ingredientsText=, ingredientsTextFr=, genericName=, imageUrl=https://images.openfoodfacts.org/images/products/800/150/500/5592/front_fr.133.400.jpg, nutriScore=d, ecoScore=c, ecoScoreValue=), ProductDetail(code=3608580065340, label=Pâte à tartiner noisettes et cacao, ingredientsText=, ingredientsTextFr=, genericName=, imageUrl=https://images.openfoodfacts.org/images/products/360/858/006/53

Pâte à tartiner Nutella noisettes et cacao Nutella Nocciolata Pâte à tartiner au cacao et noisettes Pâte à tartiner noisettes et cacao Pâte à tartiner Nutella noisettes et cacao - 825g Nocciolata Pâte à tartiner Nutella noisettes et cacao - 750g Nocciolata Bio Pâte à tartiner Pâte à tartiner chocolat noisette Nutella pâte à tartiner aux noisettes et au cacao 800g Nocciolata sans lait Pâte à tartiner Milka aux noisettes Pâte à tartiner noisette du Lot et Garonne cacao Nutella Ouf! La pâte à tartiner Pâte à tartiner Pâte à tartiner Chocolat Noisette Pâte à tartiner noisettes et cacao Biscuits Nutella B-ready x15 gaufrettes fourrées - 330g Pâte à tartiner Bio Pâte à tartiner cacao noisette Nocciolata Pâte à tartiner bio sans huile de palme

In [43]:
%use lets-plot
%use krangl


In [46]:
import kotlinx.coroutines.runBlocking

// Exemple de
runBlocking {
    val search = "pate à tartiner noisette"
    val nutriScoreApp = NutriScoreApp()
    val searchResultJson: String = nutriScoreApp.searchProductsJson(search)

    val df = DataFrame.fromJsonString(searchResultJson)
    df.head(3)


    /*val plotResult = letsPlot(searchResult)
    searchResult.plot {

    }*/

}

count page page_count page_size skip additives_tags allergens_tags brands code ecoscore_grade image_url ingredients_from_palm_oil_n nova_groups nutriments nutrition_grade_fr product_name product_name_fr quantity selected_images 1604 1 24 24 0 JsonArray(value=[en:e322]) JsonArray(value=[en:milk, en:nuts, en:soybeans]) Ferrero 3017620422003 d https://images.openfoodfacts.org/images/product... 0 4 <td style="text-align:left" title="alcohol,alcohol_100g,alcohol_serving,alcohol_unit,alcohol_value,carbohydrates,carbohydrates_100g,carbohydrates_serving,carbohydrates_unit,carbohydrates_value,carbon-footprint-from-known-ingredients_product,carbon-footprint-from-known-ingredients_serving,energy,energy-kcal,energy-kcal_100g,energy-kcal_serving,energy-kcal_unit,energy-kcal_value,energy-kcal_value_computed,energy-kj,energy-kj_100g,energy-kj_serving,energy-kj_unit,energy-kj_value,energy-kj_value_computed,energy_100g,energy_serving,energy_unit,energy_value,fat,fat_100g,fat_serving,fat_unit,fat_value,fruits-vegetables-nuts-estimate-from-ingredients_100g,fruits-vegetables-nuts-estimate-from-ingredients_serving,nova-group,nova-group_100g,nova-group_serving,nutrition-score-fr,nutrition-score-fr_100g,proteins,proteins_100g,proteins_serving,proteins_unit,proteins_value,salt,salt_100g,salt_serving,salt_unit,salt_value,saturated-fat,saturated-fat_100g,saturated-fat_serving,saturated-fat_unit,saturated-fat_value,sodium,sodium_100g,sodium_serving,sodium_unit,sodium_value,sugars,sugars_100g,sugars_serving,sugars_unit,sugars_value">alcohol,alcohol_100g,alcohol_serving,alcohol_un... e Nutella Pâte à tartiner Nutella noisettes et cacao 400 g front,ingredients,nutrition,packaging 1604 1 24 24 0 JsonArray(value=[en:e322]) JsonArray(value=[en:milk, en:nuts, en:soybeans]) Ferrero 3017620425035 d https://images.openfoodfacts.org/images/product... 0 4 <td style="text-align:left" title="carbohydrates,carbohydrates_100g,carbohydrates_serving,carbohydrates_unit,carbohydrates_value,carbon-footprint-from-known-ingredients_product,carbon-footprint-from-known-ingredients_serving,energy,energy-kcal,energy-kcal_100g,energy-kcal_serving,energy-kcal_unit,energy-kcal_value,energy-kcal_value_computed,energy-kj,energy-kj_100g,energy-kj_serving,energy-kj_unit,energy-kj_value,energy-kj_value_computed,energy_100g,energy_serving,energy_unit,energy_value,fat,fat_100g,fat_serving,fat_unit,fat_value,fiber,fiber_100g,fiber_serving,fiber_unit,fiber_value,fruits-vegetables-nuts-estimate-from-ingredients_100g,fruits-vegetables-nuts-estimate-from-ingredients_serving,nova-group,nova-group_100g,nova-group_serving,nutrition-score-fr,nutrition-score-fr_100g,proteins,proteins_100g,proteins_serving,proteins_unit,proteins_value,salt,salt_100g,salt_serving,salt_unit,salt_value,saturated-fat,saturated-fat_100g,saturated-fat_serving,saturated-fat_unit,saturated-fat_value,sodium,sodium_100g,sodium_serving,sodium_unit,sodium_value,sugars,sugars_100g,sugars_serving,sugars_unit,sugars_value">carbohydrates,carbohydrates_100g,carbohydrates_... e Nutella Nutella 1 kg front,ingredients,nutrition,packaging 1604 1 24 24 0 JsonArray(value=[en:e322, en:e322i]) JsonArray(value=[en:milk, en:nuts, en:soybeans]) Rigoni di Asiago 8001505005592 c https://images.openfoodfacts.org/images/product... 0 4 <td style="text-align:left" title="carbohydrates,carbohydrates_100g,carbohydrates_serving,carbohydrates_unit,carbohydrates_value,cocoa,cocoa_100g,cocoa_label,cocoa_serving,cocoa_unit,cocoa_value,energy,energy-kcal,energy-kcal_100g,energy-kcal_serving,energy-kcal_unit,energy-kcal_value,energy-kcal_value_computed,energy-kj,energy-kj_100g,energy-kj_serving,energy-kj_unit,energy-kj_value,energy-kj_value_computed,energy_100g,energy_serving,energy_unit,energy_value,fat,fat_100g,fat_serving,fat_unit,fat_value,fiber,fiber_100g,fiber_serving,fiber_unit,fiber_value,fruits-vegetables-nuts-estimate-from-ingredients_100g,fruits-vegetables-nuts-estimate-from-ingredients_serving,nova-group,nova-group_100g,nova-group_serving,nutritio